s[]
word=body.split(' ')
lowercase

# data

In [1]:
from os import listdir
from os.path import isfile, join
easy_ham = [f for f in listdir("data/easy_ham") if isfile(join("data/easy_ham", f))] # 2500 = 1716+784
easy_ham_2 = [f for f in listdir("data/easy_ham_2") if isfile(join("data/easy_ham_2", f))] # 1400 = 960+440
hard_ham = [f for f in listdir("data/hard_ham") if isfile(join("data/hard_ham", f))] # 250 = 172+78
spam = [f for f in listdir("data/spam") if isfile(join("data/spam", f))] # 500 = 344+156
spam_2 = [f for f in listdir("data/spam_2") if isfile(join("data/spam_2", f))] # 1396 = 958+438
for index in range(len(easy_ham)):
    easy_ham[index] = "data/easy_ham/"+ easy_ham[index]
for index in range(len(easy_ham_2)):
    easy_ham_2[index] = "data/easy_ham_2/"+ easy_ham_2[index]
for index in range(len(hard_ham)):
    hard_ham[index] = "data/hard_ham/"+ hard_ham[index]
for index in range(len(spam)):
    spam[index] = "data/spam/"+ spam[index]
for index in range(len(spam_2)):
    spam_2[index] = "data/spam_2/"+ spam_2[index]

In [2]:
ham = easy_ham + easy_ham_2 + hard_ham
spam = spam + spam_2

In [3]:
from sklearn.model_selection import train_test_split
Training_Ham, Testing_Ham = train_test_split(ham, test_size = 0.1, random_state = 1)
Training_Spam, Testing_Spam = train_test_split(spam, test_size = 0.1, random_state = 1)

In [4]:
print(len(Training_Ham), len(Testing_Ham), len(Training_Spam), len(Testing_Spam))

3735 415 1706 190


In [5]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
s=set(stopwords.words('english'))
# s=[]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
import re
import codecs
frequency_list = [1, 2, 3, 5, 10, 20, 30, 50, 100, 200, 300]
# frequency_list = [1, 2, 3, 5]
# frequency_list = [1000]

dictionary = dict()
Training = Training_Ham + Training_Spam
Testing = Testing_Ham + Testing_Spam


In [7]:
import numpy as np
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.neural_network import MLPClassifier

#----------------------------------------------- SVM -----------------------------------------------

f = open("ret/SVM_NN(Spam).txt", "a")
for frequency in frequency_list:
    print('frequency is ' + str(frequency))
    f.write('\nfrequency is ' + str(frequency) + '\n')
    dictionary = dict()
    
    for file_names in Training:
        #f = open(file_names, "r")
        ## UnicodeDecodeError: 'cp950' codec can't decode byte 0x93 in position 1885: illegal multibyte sequence
        #f = open(file_names, "r", encoding="utf-8")
        ## UnicodeDecodeError: 'utf-8' codec can't decode byte 0x93 in position 1885: invalid start byte

        file = codecs.open(file_names, 'r', encoding='utf-8', errors='ignore')

        body = file.read().split("\n\n", 1)[1].lower()
#         body = file.read()
        #word = re.findall(r"[\w']+", body) # 93783 ## 21525
        word = re.findall(r"[a-zA-Z']+", body) # 123656 ## 21224
#         token = body.split()
#         word = []
#         for string in token:
#             word += re.split(r'[`\-=~!@#$%^&*()_+\[\]{};\'\\:"|<,./<>?]', string)
        word = body.split(' ')
        for i in word:
            dictionary[i] = dictionary.get(i, 0) + 1
#     print(len(dictionary))
    dictionary = {i:dictionary[i] for i in dictionary if dictionary[i]>frequency}
#     print(len(dictionary))
    #print(dictionary)
    list_dictionary = list(dictionary)
    list_dictionary = list(filter(lambda w: not w in s,list_dictionary))
    list_dictionary
#     print(len(list_dictionary))
    
    # Remove common word in dictionary
    for s__ in s:
        if s__ in dictionary:
            del dictionary[s__]
#     print(len(dictionary))

    print('dict length: ' + str(len(dictionary)))
    f.write('dict length: ' + str(len(dictionary)))
    count = 0
    for i in dictionary:
    #     print(i)
        dictionary[i] = count
        count += 1
    dictionary

    train_ndarray = np.zeros((len(Training), len(dictionary)), dtype = int)
    train_ndarray.shape

    label_train = [0] * len(Training_Ham) + [1] * len(Training_Spam)
    label_test = [0] * len(Testing_Ham) + [1] * len(Testing_Spam)

    count = 0
    for Ham_mail in Training_Ham:
        file = codecs.open(Ham_mail, 'r', encoding='utf-8', errors='ignore')
        body = file.read().split("\n\n", 1)[1].lower()
#         body = file.read()
        word = re.findall(r"[a-zA-Z']+", body)
#         word = body.split(' ')
        for w in word:
            if w in dictionary:
                train_ndarray[count][dictionary[w]] = 1
    #     print(train_ndarray[count])
        count += 1
    for Spam_mail in Training_Spam:
        file = codecs.open(Spam_mail, 'r', encoding='utf-8', errors='ignore')
        body = file.read().split("\n\n", 1)[1].lower()
#         body = file.read()

        word = re.findall(r"[a-zA-Z']+", body)
#         word = body.split(' ')
        for w in word:
            if w in dictionary:
                train_ndarray[count][dictionary[w]] = 1
        count += 1
    train_dataframe = pd.DataFrame(train_ndarray)

    test_ndarray = np.zeros((len(Testing), len(dictionary)), dtype = int)
    test_ndarray.shape

    count = 0
    for Ham_mail in Testing_Ham:
        file = codecs.open(Ham_mail, 'r', encoding='utf-8', errors='ignore')
        body = file.read().split("\n\n", 1)[1].lower()
#         body = file.read()
        word = re.findall(r"[a-zA-Z']+", body)
#         word = body.split(' ')
        for w in word:
            if w in dictionary:
                test_ndarray[count][dictionary[w]] = 1
    #     print(train_ndarray[count])
        count += 1
    for Spam_mail in Testing_Spam:
        file = codecs.open(Spam_mail, 'r', encoding='utf-8', errors='ignore')
        body = file.read().split("\n\n", 1)[1].lower()
#         body = file.read()
        word = re.findall(r"[a-zA-Z']+", body)
#         word = body.split(' ')
        for w in word:
            if w in dictionary:
                test_ndarray[count][dictionary[w]] = 1
        count += 1
    test_dataframe = pd.DataFrame(test_ndarray)

    SVM = SVC(kernel='linear', probability=True)
    SVM.fit(train_dataframe, label_train)

    SVM_result = SVM.predict(test_dataframe)
#     SVM_result_prob = SVM.predict_proba(test_dataframe)
    
#----------------------------------------------- NN -----------------------------------------------
    dictionary = dict()
    
    for file_names in Training:
        #f = open(file_names, "r")
        ## UnicodeDecodeError: 'cp950' codec can't decode byte 0x93 in position 1885: illegal multibyte sequence
        #f = open(file_names, "r", encoding="utf-8")
        ## UnicodeDecodeError: 'utf-8' codec can't decode byte 0x93 in position 1885: invalid start byte

        file = codecs.open(file_names, 'r', encoding='utf-8', errors='ignore')

        body = file.read().split("\n\n", 1)[1].lower()
        #word = re.findall(r"[\w']+", body) # 93783 ## 21525
        word = re.findall(r"[a-zA-Z']+", body) # 123656 ## 21224
        for i in word:
            dictionary[i] = dictionary.get(i, 0) + 1
#     print(len(dictionary))
    dictionary = {i:dictionary[i] for i in dictionary if dictionary[i]>frequency}
#     print(len(dictionary))
    #print(dictionary)
    list_dictionary = list(dictionary)
    list_dictionary = list(filter(lambda w: not w in s,list_dictionary))
    list_dictionary
#     print(len(list_dictionary))
    
    # Remove common word in dictionary
    for s__ in s:
        if s__ in dictionary:
            del dictionary[s__]
#     print(len(dictionary))

#     print('dict length: ' + str(len(dictionary)))
#     f.write('dict length: ' + str(len(dictionary)))
    count = 0
    for i in dictionary:
    #     print(i)
        dictionary[i] = count
        count += 1
    dictionary

    train_ndarray = np.zeros((len(Training), len(dictionary)), dtype = int)
    train_ndarray.shape

    label_train = [0] * len(Training_Ham) + [1] * len(Training_Spam)
    label_test = [0] * len(Testing_Ham) + [1] * len(Testing_Spam)

    count = 0
    for Ham_mail in Training_Ham:
        file = codecs.open(Ham_mail, 'r', encoding='utf-8', errors='ignore')
        body = file.read().split("\n\n", 1)[1].lower()
        word = re.findall(r"[a-zA-Z']+", body)
        for w in word:
            if w in dictionary:
                train_ndarray[count][dictionary[w]] = 1
    #     print(train_ndarray[count])
        count += 1
    for Spam_mail in Training_Spam:
        file = codecs.open(Spam_mail, 'r', encoding='utf-8', errors='ignore')
        body = file.read().split("\n\n", 1)[1].lower()
        word = re.findall(r"[a-zA-Z']+", body)
        for w in word:
            if w in dictionary:
                train_ndarray[count][dictionary[w]] = 1
        count += 1
    train_dataframe = pd.DataFrame(train_ndarray)

    test_ndarray = np.zeros((len(Testing), len(dictionary)), dtype = int)
    test_ndarray.shape

    count = 0
    for Ham_mail in Testing_Ham:
        file = codecs.open(Ham_mail, 'r', encoding='utf-8', errors='ignore')
        body = file.read().split("\n\n", 1)[1].lower()
        word = re.findall(r"[a-zA-Z']+", body)
        for w in word:
            if w in dictionary:
                test_ndarray[count][dictionary[w]] = 1
    #     print(train_ndarray[count])
        count += 1
    for Spam_mail in Testing_Spam:
        file = codecs.open(Spam_mail, 'r', encoding='utf-8', errors='ignore')
        body = file.read().split("\n\n", 1)[1].lower()
        word = re.findall(r"[a-zA-Z']+", body)
        for w in word:
            if w in dictionary:
                test_ndarray[count][dictionary[w]] = 1
        count += 1
    test_dataframe = pd.DataFrame(test_ndarray)

    NN = MLPClassifier()
    NN.fit(train_dataframe, label_train)

    NN_result = NN.predict(test_dataframe)
    NN_result_test = NN_result.copy()
    NN_result_prob = NN.predict_proba(test_dataframe)
    
    for i in range(len(NN_result)):
        if (SVM_result[i] == 1):
            NN_result_test[i] = 1
            if (NN_result_prob[i][0] > 0.4) and (NN_result_prob[i][1] > 0.4):
                NN_result[i] = 1
#             print(i)

    
    print('\n----------------- SVM_NN(Spam) -------------------\n')
    f.write('\n----------------- SVM_NN(Spam) -------------------\n')
    
    tn, fp, fn, tp = confusion_matrix(label_test, NN_result).ravel()

    Accuracy = (tp+tn)/(tn+ fp+ fn+ tp)
    spam_Precision = (tp)/(fp+ tp)
    spam_Recall = (tp)/(fn+ tp)
    
    ham_Precision = (tn)/(fn + tn)
    ham_Recall = (tn)/(fp + tn)
    
    print('Spam\nRecall\t\tPrecision\tAccuracy')
    f.write('\n\nSpam\nRecall\t\tPrecision\tAccuracy\n')
    print(str(round(spam_Recall*100, 2))+'\t\t'+ str(round(spam_Precision*100, 2))+'\t\t'+str(round(Accuracy*100, 2)))
    f.write(str(round(spam_Recall*100, 2))+'\t\t'+ str(round(spam_Precision*100, 2))+'\t\t'+str(round(Accuracy*100, 2)))
    
    print('\nHam\nRecall\t\tPrecision\tAccuracy')
    f.write('\n\nHam\nRecall\t\tPrecision\tAccuracy\n')
    print(str(round(ham_Recall*100, 2))+'\t\t'+ str(round(ham_Precision*100, 2))+'\t\t'+str(round(Accuracy*100, 2)))
    f.write(str(round(ham_Recall*100, 2))+'\t\t'+ str(round(ham_Precision*100, 2))+'\t\t'+str(round(Accuracy*100, 2))+ '\n\n')
    
    print(tn, fp, fn, tp)
    f.write('tn ' + str(tn) + ' fp ' + str(fp) + ' fn ' + str(fn) + ' tp ' + str(tp))
    print('\n------------------------------------\n')
    f.write('\n------------------------------------\n')
    
    print('\n----------------- test -------------------\n')
    f.write('\n----------------- test -------------------\n')
    
    tn, fp, fn, tp = confusion_matrix(label_test, NN_result_test).ravel()

    Accuracy = (tp+tn)/(tn+ fp+ fn+ tp)
    spam_Precision = (tp)/(fp+ tp)
    spam_Recall = (tp)/(fn+ tp)
    
    ham_Precision = (tn)/(fn + tn)
    ham_Recall = (tn)/(fp + tn)
    
    print('Spam\nRecall\t\tPrecision\tAccuracy')
    f.write('\n\nSpam\nRecall\t\tPrecision\tAccuracy\n')
    print(str(round(spam_Recall*100, 2))+'\t\t'+ str(round(spam_Precision*100, 2))+'\t\t'+str(round(Accuracy*100, 2)))
    f.write(str(round(spam_Recall*100, 2))+'\t\t'+ str(round(spam_Precision*100, 2))+'\t\t'+str(round(Accuracy*100, 2)))
    
    print('\nHam\nRecall\t\tPrecision\tAccuracy')
    f.write('\n\nHam\nRecall\t\tPrecision\tAccuracy\n')
    print(str(round(ham_Recall*100, 2))+'\t\t'+ str(round(ham_Precision*100, 2))+'\t\t'+str(round(Accuracy*100, 2)))
    f.write(str(round(ham_Recall*100, 2))+'\t\t'+ str(round(ham_Precision*100, 2))+'\t\t'+str(round(Accuracy*100, 2))+ '\n\n')
    
    print(tn, fp, fn, tp)
    f.write('tn ' + str(tn) + ' fp ' + str(fp) + ' fn ' + str(fn) + ' tp ' + str(tp))
    print('\n------------------------------------\n')
    f.write('\n------------------------------------\n')
    
f.close()

frequency is 1
dict length: 83608

----------------- SVM_NN(Spam) -------------------

Spam
Recall		Precision	Accuracy
97.89		97.89		98.68

Ham
Recall		Precision	Accuracy
99.04		99.04		98.68
411 4 4 186

------------------------------------


----------------- test -------------------

Spam
Recall		Precision	Accuracy
97.89		97.89		98.68

Ham
Recall		Precision	Accuracy
99.04		99.04		98.68
411 4 4 186

------------------------------------

frequency is 2
dict length: 48665

----------------- SVM_NN(Spam) -------------------

Spam
Recall		Precision	Accuracy
97.89		97.89		98.68

Ham
Recall		Precision	Accuracy
99.04		99.04		98.68
411 4 4 186

------------------------------------


----------------- test -------------------

Spam
Recall		Precision	Accuracy
98.42		97.4		98.68

Ham
Recall		Precision	Accuracy
98.8		99.27		98.68
410 5 3 187

------------------------------------

frequency is 3
dict length: 36475

----------------- SVM_NN(Spam) -------------------

Spam
Recall		Precision	Accuracy

In [8]:
frequency is 5
dict length: 25515
Recall		Precision	Accuracy
99.83		93.96		97.94

SyntaxError: invalid syntax (<ipython-input-8-c1aaeb87bc00>, line 2)

In [ ]:
NB = MultinomialNB()
NB.fit(train_dataframe, label_train)

NB_result = NB.predict_proba(test_dataframe)